(results)=
# Results

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters

In [ ]:
ships = ['wpcc','LNG','kvlcc2_hsva']

In [ ]:
%reload_kedro

def load_model_tests_ship(ship = 'wpcc'):
    
    model_tests = model_test_ids[ship]
    
    dfs = []
    
    for id in model_tests:
        
        df_ = catalog.load(f"{ship}.{id}.data")

            
        df_['id'] = id
        dfs.append(df_)
            
    df = pd.concat(dfs)
    
    return df


In [ ]:
def select_model_tests(df, n=9):
    df['|psi|'] = df['psi'].abs()
    
    indexes = df.groupby(by='id')['|psi|'].max().sort_values(ascending=False).index[0:n]
    mask = df['id'].isin(indexes)
    df_select = df[mask]
    return df_select

In [ ]:
def plot_ship(df_select, ship, vmm="vmm_abkowitz"):
    
    fig,axes = plt.subplots(ncols=3, nrows=3)
    fig.set_size_inches(14,10)
    axes = axes.flatten()
    for ax, (id, df_) in zip(axes,df_select.groupby(by='id')):
        
        prediction = catalog.load(f"{ship}.updated.{vmm}.joined.{id}.data_resimulate")

        mask = df_['x0'] < 60
        df_ = df_.loc[mask]
        mask = prediction['x0'] < 60
        prediction = prediction.loc[mask]
        
        
        dataframes = {
            'model test':df_,
            'prediction':prediction,
            
        }
        
        track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], ax=ax, N=4)
        ax.get_legend().set_visible(False)
        ax.set_title(None)
    
    axes[0].legend()
    fig.suptitle(f'Track plots: {ship}')
    plt.tight_layout()
    return fig
    

In [ ]:
for ship in ships:
    
    df = load_model_tests_ship(ship=ship)
    ship_data = catalog.load(f"{ship}.ship_data")
    df_select = select_model_tests(df=df)
    fig = plot_ship(df_select, ship, vmm='vmm_abkowitz')
    glue(ship, fig, display=False)

## Simulations

Comparisons of model test and corresponding simulations with identified Abkowitz model can be seen for three test cases in [fig](wpcc), [fig](LNG) and [fig](kvlcc2_hsva).

```{glue:figure} wpcc
:figwidth: 1000px
:name: "wpcc"

Comparison of model tests and simulations with identified Abkowitz model for WPCC.
```

```{glue:figure} LNG
:figwidth: 1000px
:name: "LNG"

Comparison of model tests and simulations with identified Abkowitz model for LNG.
```

```{glue:figure} kvlcc2_hsva
:figwidth: 1000px
:name: "kvlcc2_hsva"

Comparison of model tests and simulations with identified Abkowitz model for KVLCC2.
```



## Discussion

In [ ]:
regression_abkowitz = catalog.load("kvlcc2_hsva.updated.vmm_abkowitz.joined.regression")
parameters = pd.DataFrame()
parameters['mean'] = regression_abkowitz.parameters['regressed']

std = dict(regression_abkowitz.model_X.bse)
std.update(regression_abkowitz.model_Y.bse)
std.update(regression_abkowitz.model_N.bse)
std = pd.Series(std)

parameters['std'] = std
quantile_cuts = [0.5, 0.67, 0.70, 0.75]
fig = plot_parameters(parameters, quantile_cuts=quantile_cuts);
fig.set_size_inches(5.9055,10)
plt.tight_layout()
glue("parameters_abkowits", fig, display=False)

import seaborn as sns

model = regression_abkowitz.model_Y

cov = model.cov_HC0

diag = np.sqrt(np.diag(np.diag(cov)))
gaid = np.linalg.inv(diag)
corr = gaid @ cov @ gaid
corr = np.tril(corr)
corr = pd.DataFrame(corr, index=model.params.keys(), columns=model.params.keys())

fig,ax=plt.subplots()
grid = sns.heatmap(corr.abs(), cmap='gray_r')
glue("heatmap_parameters", grid.figure, display=False)

In [ ]:
regression_simple = catalog.load("kvlcc2_hsva.updated.vmm_martins_simple.joined.regression")
parameters = pd.DataFrame()
parameters['mean'] = regression_simple.parameters['regressed']

std = dict(regression_simple.model_X.bse)
std.update(regression_simple.model_Y.bse)
std.update(regression_simple.model_N.bse)
std = pd.Series(std)

parameters['std'] = std
quantile_cuts = [0.7]
fig = plot_parameters(parameters, quantile_cuts=quantile_cuts);
glue("parameters_martins_simple", fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(7,5)
plt.text(0.01, 0.05, str(regression_abkowitz.model_Y.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
glue('regression_abkowitz_summary', fig, display=False)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(7,5)
plt.text(0.01, 0.05, str(regression_simple.model_Y.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
glue('regression_simple_summary', fig, display=False)

The printout from the OLS Regression results for Abkowitz fy-equation for KVLCC2 data is shown in [fig](regression_abkowitz_summary). The printout shows the regressed parameters (coef) together with, standard deviation (std err) and confidence intervals ([0.025 	0.975]). The P-values from hypothesis tests (P>|t|) is very high for some of the parameters such as: $Y_{\delta}$, $Y_r$ etc. This is indicating that the model has too many parameters for the present dataset. Strong multicollinearity is also reported which is a known problem for the the VMM:s {cite:p}`luo_parameter_2016`, {cite:p}`wang_quantifying_2018`. The coefficients which are found do not
have to be physically correct, but mathematically correct {cite:p}`ittc_maneuvering_2008` which is also something that is shown here, where some of the parameters have very large values (in the order of 10^8). This means that the regressed Abkowitz model for KVLCC2 does a very good job in describing the known data, that we see here, but will not work very well when extrapolating outside the known data, when making predictions.

```{glue:figure} regression_abkowitz_summary
:name: "regression_abkowitz_summary"

Regression result for Abkowitz model fy equation on KVLCC2 data. 
```

Printout from an OLS Regression for a simplified Abkowitz fy-equation for KVLCC2 data is shown in [fig](regression_simple_summary). This model has almost as high accuracy as the full Abkowitz model when comparing the R^2 values (0.996 compared to 0.993). The coefficients in this model also have much smaller values and smaller P-values. So this means that the simplified model describes the known data almost as good as the full Abkowitz model, but will most likely work better when making predictions outside the known data.

```{glue:figure} regression_simple_summary
:name: "regression_simple_summary"

Regression result for simplified Abkowitz model fy equation on KVLCC2 data. 
```

The identified parameters of an Abkowitz model for KVLCC2 are shown in [figure](parameters_abkowits). 
It can be seen that the uncertainty (standard deviation of the estimated parameters), indicated by the black error bars, is very high for some of the parameters.
It can also be seen that some of the parameters have very high values, in the order of 10^8. 

Identifying parameters of a simplified Abkowitz model for KVLCC2 gives much smaller parameters and lower uncertainty as seen in [figure](parameters_martins_simple).

```{glue:figure} parameters_abkowits
:figwidth: 1000px
:name: "parameters_abkowits"

Identified parameters in Abkowitz model for KVLCC2.
```

```{glue:figure} parameters_martins_simple
:figwidth: 1000px
:name: "parameters_martins_simple"

Identified parameters in simplified Abkowitz model for KVLCC2.
```

Can the regressed Abkowitz model be trusted considering the high uncertainty in the parameter values? 

This can be investigated using Monte Carlo simulations with alternative realizations of the regression. 
Manoeuvre simulation can be performed with parameter sets taken as random samples from a multivariate gausian distribution defined by the mean values and 
covariance matrix from the linear regression.

The covariance matrix needs to be included in the Monte Carlo simulations since there is high correlation between some of the parameters as seen in [fig](heatmap_parameters).

```{glue:figure} heatmap_parameters
:figwidth: 1000px
:name: "heatmap_parameters"

Absolute correlation coefficient of identified parameters in Abkowitz model for KVLCC2.
```

In [ ]:
from scipy.linalg import block_diag
from scipy.stats import norm, multivariate_normal
import tqdm
import sys
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
regression = regression_abkowitz
means = pd.concat([regression.model_X.params,regression.model_Y.params,regression.model_N.params])
covs = [regression.model_X.cov_HC0,regression.model_Y.cov_HC0,regression.model_N.cov_HC0]

std = regression.model_X.bse.copy()
std=std.append(regression.model_Y.bse)
std=std.append(regression.model_N.bse)

cov = block_diag(*covs)

rv = multivariate_normal(mean=means.values, cov=cov, allow_singular=True)
np.random.seed(42)
N_=10
df_parameter_variation = pd.DataFrame(data=rv.rvs(N_), columns=means.index)

In [ ]:
#for key in means.keys():
#
#    rv = norm(loc=means[key], scale=std[key])
#    
#    xs = np.linspace(df_parameter_variation[key].min(), df_parameter_variation[key].max(),100)
#    
#    grid = sns.displot(data=df_parameter_variation[key], kind='hist', stat="density")
#    
#    ax = grid.ax
#    ax.plot(xs, rv.pdf(xs));



In [ ]:
def monte_carlo(data_smooth, df_parameter_variation, model, ek):

    dataframes = {}
    with tqdm.tqdm(total=len(df_parameter_variation), file=sys.stdout) as pbar:
        for index, parameters_ in df_parameter_variation.iterrows():
            model_ = model.copy()
            model_.parameters.update(parameters_)
            
            df_ = simulate_euler(data=data_smooth, model=model_,ek=ek, solver='Radau') 
            dataframes[index] = df_
        
            pbar.update(1)
        
    return dataframes

In [ ]:
#vmm_name = 'vmm_abkowitz_simple'
#vmm_name = 'vmm_martins_simple'
vmm_name = 'vmm_abkowitz'
#id = 'HSVA_CPMC_KVLCC2_Z_15_05'
id = 'HSVA_CPMC_KVLCC2_Z_25_05'

model = catalog.load(f"kvlcc2_hsva.updated.{ vmm_name }.joined.model")
ek = catalog.load(f"kvlcc2_hsva.{ vmm_name }.ek")

#id = 22774
#regression = catalog.load(f"{ vmm_name }.motion_regression.no_circle.regression")
df_smooth = catalog.load(f"kvlcc2_hsva.updated.{ id }.data_ek_smooth")
ship = "kvlcc2_hsva"
df_sim = catalog.load(f"{ ship }.updated.{ vmm_name }.joined.{ id }.data_resimulate")



In [ ]:
model.turning_circle(

In [ ]:
dataframes = monte_carlo(df_smooth, df_parameter_variation, model=model, ek=ek)

In [ ]:
dataframes['model test'] = df_smooth
dataframes['VMM all'] = df_sim

In [ ]:
styles={'model test':{'style':'k-','lw':2},
       'VMM all':{'style':'r-','lw':2},
        
       }

for index, parameters_ in df_parameter_variation.iterrows():
    styles[index] = {'style':'b-', 
                     'alpha':0.1,
                     'label':'_Hidden label'}

#for dataframes_ in displays:
#    fig,ax=plt.subplots()
#    grid.fig.set_size_inches(0.7*np.array(matplotlib.rcParams["figure.figsize"]))
#    
#    ax = track_plots(dataframes_, lpp=ship_data['L'], beam=ship_data['B'], plot_boats=False, flip=True, N=7, styles=styles, ax=ax)
#    #ax.set_xlim(0,25)
#    #ax.set_ylim(-20,5)   

In [ ]:
fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], plot_boats=False, flip=True, N=7, styles=styles, ax=ax)
   